In [1]:
import math
import random
import pandas as pd
import numpy as np
import tkinter as tk


from PIL import ImageTk, Image
from scoring_frames import *
from spades import *
from spades_apr_4 import *
from spades_logic_2 import *

--------------------------------------------------------------------------------------------------------------
## Let's initialize this with
# Bids
## and
# A score_frame
--------------------------------------------------------------------------------------------------------------

In [2]:
deck_bids = initialize_spades(spades_deal(shuffled(spades_deck())))
p1_bid = deck_bids[2]
p2_bid = deck_bids[4]
p3_bid = deck_bids[6]
p4_bid = deck_bids[8]
        
score_frame = pd.DataFrame({"Bids":[p1_bid, p2_bid, p3_bid, p4_bid,\
    "nan"],"Wins":[0, 0, 0, 0, 0], "Bags":[0, 0, 0, 0, 0], "Score":[0,0,0,0,0]}, index = ["Player 1", "Player 2", "Player 3", "Player 4","Winner"])
score_frame

Bids  Wins  Bags  Score
Player 1    2     0     0      0
Player 2    2     0     0      0
Player 3    3     0     0      0
Player 4    4     0     0      0
Winner    nan     0     0      0


 --------------------------------------------------------------------------------------------------------------
# Now let's create a function to update the frame with rounds
 --------------------------------------------------------------------------------------------------------------

In [3]:
def add_frame(n, df, p1_disc, p2_disc, p3_disc, p4_disc, winner):
    frame = pd.DataFrame({"Hand " + str(n):[p1_disc, p2_disc, p3_disc, p4_disc,\
    winner ]}, index = ["Player 1", "Player 2", "Player 3", "Player 4","Winner"])
    winner = frame["Hand " + str(n)].loc["Winner"]
    df["Wins"]["Player " + str(winner)] += 1

    
    return_frame = pd.concat([df, frame], axis = 1)
    return return_frame
                         
test = add_frame(1, score_frame, 1, 2, 3, 4, 1)
test

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Bids  Wins  Bags  Score  Hand 1
Player 1    2     1     0      0       1
Player 2    2     0     0      0       2
Player 3    3     0     0      0       3
Player 4    4     0     0      0       4
Winner    nan     0     0      0       1

-------------------------------------------------------------------------------------------------------------------------------
# Let's try adding rows in a loop 
-------------------------------------------------------------------------------------------------------------------------------

In [4]:
for num in range(1,14):
    if num == 1:
        loop_test = add_frame(num, score_frame, 1 * num, 1 * num, 1 * num, 1 * num, 2)
        
    if num > 1:
        loop_test = add_frame(num, loop_test, 1 * num, 1 * num, 1 * num, 1 * num, 3)
    

loop_test

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Bids  Wins  Bags  Score  Hand 1  Hand 2  Hand 3  Hand 4  Hand 5  \
Player 1    2     1     0      0       1       2       3       4       5   
Player 2    2     1     0      0       1       2       3       4       5   
Player 3    3    12     0      0       1       2       3       4       5   
Player 4    4     0     0      0       1       2       3       4       5   
Winner    nan     0     0      0       2       3       3       3       3   

          Hand 6  Hand 7  Hand 8  Hand 9  Hand 10  Hand 11  Hand 12  Hand 13  
Player 1       6       7       8       9       10       11       12       13  
Player 2       6       7       8       9       10       11       12       13  
Player 3       6       7       8       9       10       11       12       13  
Player 4       6       7       8       9       10       11       12       13  
Winner         3       3       3       3        3        3        3        3

-------------------------------------------------------------------------------------------------------------------------------
# Now let's create a function to calculate the round score
# and add it to the loop_test dataframe
-------------------------------------------------------------------------------------------------------------------------------

In [5]:
def add_calculation(df):
    for num in range(1,3):
        if num == 1:
            mate = 3
        if num == 2:
            mate = 4
        
        wins = df["Wins"]["Player " + str(num)]
        bids = df["Bids"]["Player " + str(num)]
        bags = wins - bids
        
        mate_wins = df["Wins"]["Player " + str(mate)]
        mate_bids = df["Bids"]["Player " + str(mate)]
        mate_bags = mate_wins - mate_bids
        
        team_wins = wins + mate_wins
        team_bids = bids + mate_bids
        team_bags = bags + mate_bags
        
        
        if team_wins < team_bids and bids != 0 and mate_bids != 0:
            df["Score"]["Player " + str(num)] -= 50
            continue
            
        if team_wins >= team_bids and bids != 0 and mate_bids != 0:
            df["Score"]["Player " + str(num)] +=  team_bids * 10
            df["Bags"]["Player " + str(num)] += team_bags
            continue
            
        if bids == 0 and team_wins >= team_bids:
            if wins > 0:
                df["Score"]["Player " + str(num)] +=  (team_bids * 10) - 100
                df["Bags"]["Player " + str(num)] += team_bags
                
            if wins == 0:
                df["Score"]["Player " + str(num)] +=  (team_bids * 10) + 100
                df["Bags"]["Player " + str(num)] += team_bags
                            
        if mate_bids == 0 and team_wins >= team_bids:
            if mate_wins > 0:
                df["Score"]["Player " + str(num)] +=  (team_bids * 10) - 100
                df["Bags"]["Player " + str(num)] += (wins + mate_wins) - (bids + mate_bids)
                
            if mate_wins == 0:
                df["Score"]["Player " + str(num)] +=  (team_bids * 10) + 100
                df["Bags"]["Player " + str(num)] += team_bags
                
        if bids == 0 and team_wins <= team_bids:
            if wins > 0:
                df["Score"]["Player " + str(num)] -=  150
                
            if wins == 0:
                df["Score"]["Player " + str(num)] +=  50
                            
        if mate_bids == 0 and team_wins <= team_bids:
            if mate_wins > 0:
                df["Score"]["Player " + str(num)] -=  150
                
            if mate_wins == 0:
                df["Score"]["Player " + str(num)] -=  150
            
    return df
            

In [6]:
wins = add_calculation(loop_test)
wins

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Bids  Wins  Bags  Score  Hand 1  Hand 2  Hand 3  Hand 4  Hand 5  \
Player 1    2     1     8     50       1       2       3       4       5   
Player 2    2     1     0    -50       1       2       3       4       5   
Player 3    3    12     0      0       1       2       3       4       5   
Player 4    4     0     0      0       1       2       3       4       5   
Winner    nan     0     0      0       2       3       3       3       3   

          Hand 6  Hand 7  Hand 8  Hand 9  Hand 10  Hand 11  Hand 12  Hand 13  
Player 1       6       7       8       9       10       11       12       13  
Player 2       6       7       8       9       10       11       12       13  
Player 3       6       7       8       9       10       11       12       13  
Player 4       6       7       8       9       10       11       12       13  
Winner         3       3       3       3        3        3        3        3

# Alas, we have two functions to manipulate a data frame during a round
# and calculate a score at the end of a round
### add_frame(n, df, p1_disc, p2_disc, p3_disc, p4_disc, winner)
### add_calculation(df)







# Next we will create some cards with and some attibutes for those cards
# depending on how they are dealt

In [34]:
class Cards:
    
    def __init__(self):
        deck = initialize_spades(spades_deal(shuffled(spades_deck())))
        self.turn = deck[0]
        
        self.p1 = deck[1]
        self.p1_bid = deck[2]
        
        
        self.p2 = deck[3]
        self.p2_bid = deck[4]
                
        
        self.p3 = deck[5]
        self.p3_bid = deck[6]
        
        
        self.p4 = deck[7]
        self.p4_bid = deck[8]
                
        self.team1_bids = self.p1_bid, self.p3_bid
        self.team2_bids = self.p2_bid, self.p4_bid
        
        self.suits = {"Hearts", "Clubs", "Diamonds", "Spades"}
        
        
    def hand_call(self, p_hand):
        enums = []
        for enum, face, suit in p1:
            enums.append(str(enum))
        return enums
                
    def hearts(self, p_hand):
        count = counter(p_hand)
        return count[0]
        
    def clubs(self, p_hand):
        count = counter(p_hand)
        return count[1]
        
    def diamonds(self, p_hand):
        count = counter(p_hand)
        return count[2]
    
    def spades(self, p_hand):
        count = counter(p_hand)
        return count[3]


In [35]:
for i in range(1,20):    
    teste = Cards()
    p1 = teste.p4
    p1_bid = teste.p4_bid
    
    turn = teste.turn
    
    hearts = teste.hearts(p1)
    clubs = teste.clubs(p1)
    diamonds = teste.diamonds(p1)
    spades = teste.spades(p1)
    
    total = hearts + clubs + diamonds + spades
    
    hand_call = teste.hand_call(p1)
    
    
    print("Turn=", turn, "Total=", total,  "Counts =", hearts, clubs, diamonds, spades,"\n")
    print("Hand call =", hand_call)

Turn= 3 Total= 13 Counts = 4 4 2 3 

Hand call = ['2', '6', '7', '13', '14', '15', '19', '22', '30', '35', '40', '46', '51']
Turn= 2 Total= 13 Counts = 6 2 3 2 

Hand call = ['2', '6', '7', '9', '12', '13', '14', '18', '32', '38', '39', '40', '48']
Turn= 2 Total= 13 Counts = 2 4 5 2 

Hand call = ['7', '11', '19', '22', '24', '26', '27', '29', '32', '33', '36', '44', '50']
Turn= 4 Total= 13 Counts = 6 3 1 3 

Hand call = ['1', '4', '5', '8', '10', '13', '14', '16', '25', '32', '44', '46', '50']
Turn= 1 Total= 13 Counts = 3 4 0 6 

Hand call = ['4', '6', '10', '18', '19', '22', '25', '44', '45', '48', '49', '50', '51']
Turn= 1 Total= 13 Counts = 4 3 2 4 

Hand call = ['5', '9', '10', '12', '14', '25', '26', '28', '32', '49', '50', '51', '52']
Turn= 1 Total= 13 Counts = 3 0 5 5 

Hand call = ['4', '9', '11', '27', '28', '33', '36', '39', '41', '46', '48', '50', '51']
Turn= 1 Total= 13 Counts = 2 5 2 4 

Hand call = ['2', '8', '14', '17', '21', '22', '25', '29', '30', '40', '42', '46', '4

# Below, a class is created for determining the winner

In [36]:
class Discard(Cards):
    def __init__(self):
        Cards.__init__(self)
    
    
    def winner(self, p1_disc, p2_disc, p3_disc, p4_disc, set_suit):
        for a, b, c in sorted(set([p1_disc, p2_disc, p3_disc, p4_disc]), reverse = True):
            if c == set_suit or c == "Spades":
                if (a,b,c) == p1_disc:
                    winner = 1
                if(a,b,c) == p2_disc:
                    winner = 2
                if(a,b,c) == p3_disc:
                    winner = 3
                if(a,b,c) == p4_disc:
                    winner = 4
                return winner


In [37]:
discard_test = Discard()
p1_t = (1,2,"Hearts")
p2_t = (2,3,"Hearts")
p3_t = (3,4,"Hearts")
p4_t = (52,"Ace","Spades")
suit = "Hearts"
discard_test.winner(p1_t, p2_t, p3_t, p4_t, suit)

4

In [38]:
discard_test = Discard()
p1_t = (52,"Ace","Spades")
p2_t = (2,3,"Hearts")
p3_t = (3,4,"Hearts")
p4_t = (13,"King","Hearts")
suit = "Hearts"
discard_test.winner(p1_t, p2_t, p3_t, p4_t, suit)

1